In [1]:
import datetime as dt
import pandas as pd
import numpy as np

In [15]:
educacion = pd.read_csv('/home/luupesado/7506_Datos/2018/datos_tp/joins/educacion.csv')
genero_y_edad = pd.read_csv('/home/luupesado/7506_Datos/2018/datos_tp/joins/genero_y_edad.csv')

Limpiamos el set de datos de educación para quedarnos con el mayor nivel educativo de cada postulante, tal cual lo hicimos en el tp 1

In [16]:
educacion['nombre'] = educacion['nombre'].astype('string')

In [17]:
import unicodedata
def elimina_tildes(s):
   return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

In [18]:
educacion['nombre'] = educacion['nombre'].apply(lambda x: elimina_tildes(x.decode('utf-8')))

In [19]:
educacion['nombre-estado'] = educacion['nombre'] + ' - ' + educacion['estado']

In [20]:
educacion['nombre-estado'] = educacion['nombre-estado'].astype('category')

In [21]:
categories_order = ['Otro - Abandonado',
 'Otro - En Curso',                                        
 'Otro - Graduado', 
'Secundario - Abandonado',
 'Secundario - En Curso',
 'Secundario - Graduado',
 'Terciario/Tecnico - Abandonado',
 'Terciario/Tecnico - En Curso',
 'Terciario/Tecnico - Graduado',
 'Universitario - Abandonado',
 'Universitario - En Curso',
 'Universitario - Graduado',
'Posgrado - Abandonado',
 'Posgrado - En Curso',
 'Posgrado - Graduado',
 'Master - Abandonado',
 'Master - En Curso',
 'Master - Graduado',
'Doctorado - Abandonado',
 'Doctorado - En Curso',
 'Doctorado - Graduado']

In [22]:
educacion['nombre-estado'] = educacion['nombre-estado'].cat \
                                        .set_categories(categories_order, ordered = True)

In [23]:
educacion = educacion.sort_values('nombre-estado', ascending= False).drop_duplicates(subset='idpostulante', keep = 'first')

Ahora limpiamos el set de datos de genero y edad, tal cual lo hicims el el tp1

In [24]:
import datetime as dt

In [46]:
def strdate_to_age(string):
    if string == 'nan':
        return int(np.random.rand()*100)
    date = dt.datetime.strptime(string, "%Y-%m-%d")
    today = dt.datetime(2018, 4, 1)
    edad = today.year - date.year
    if today.month >= date.month and today.day >= date.day:
        edad = edad +1
    return edad


In [14]:
genero_y_edad = genero_y_edad.dropna()

In [45]:
int(np.random.rand()*100)

21

In [47]:
genero_y_edad['edad'] = genero_y_edad['fechanacimiento'].apply(lambda x: strdate_to_age(str(x)))

In [48]:
genero_y_edad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780020 entries, 0 to 780019
Data columns (total 5 columns):
Unnamed: 0         780020 non-null int64
idpostulante       780020 non-null object
fechanacimiento    745747 non-null object
sexo               780020 non-null object
edad               780020 non-null int64
dtypes: int64(2), object(3)
memory usage: 29.8+ MB


Ahora hago el merge

In [49]:
educacion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 1070867 to 641978
Data columns (total 5 columns):
Unnamed: 0       447909 non-null int64
idpostulante     447909 non-null object
nombre           447909 non-null object
estado           447909 non-null object
nombre-estado    447909 non-null category
dtypes: category(1), int64(1), object(3)
memory usage: 17.5+ MB


In [50]:
genero_y_edad.tail()

,Unnamed: 0,idpostulante,fechanacimiento,sexo,edad
780015,297742,MVrow99,1999-09-27,FEM,19
780016,297743,dYjl8Mq,NaN,NO_DECLARA,49
780017,297744,RzrDOmR,NaN,NO_DECLARA,17
780018,297745,X95z68O,1998-10-29,MASC,20
780019,297746,vVjoqbj,NaN,NO_DECLARA,32


In [51]:
del genero_y_edad['Unnamed: 0']
del educacion['Unnamed: 0']

In [59]:
postulantes = pd.merge(genero_y_edad, educacion, on='idpostulante', how='outer')

In [60]:
postulantes.head()

,idpostulante,fechanacimiento,sexo,edad,nombre,estado,nombre-estado
0,6MM,1985-01-01,MASC,34,NaN,NaN,NaN
1,6MM,1985-01-01,MASC,34,NaN,NaN,NaN
2,ZX1,NaN,NO_DECLARA,56,NaN,NaN,NaN
3,ZX1,NaN,NO_DECLARA,94,NaN,NaN,NaN
4,YZP,1961-05-01,MASC,57,Master,En Curso,Master - En Curso


In [61]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 780020 entries, 0 to 780019
Data columns (total 7 columns):
idpostulante       780020 non-null object
fechanacimiento    745747 non-null object
sexo               780020 non-null object
edad               780020 non-null int64
nombre             707486 non-null object
estado             707486 non-null object
nombre-estado      707486 non-null category
dtypes: category(1), int64(1), object(5)
memory usage: 42.4+ MB


Ahora filtramos los postulantes por edad, que no habíamos olvidade de hacer antes del merge. NOs quedamos con los postulantes entre 16 y 70 años. Según vimos en el primer TP había muy pocos datos de gente de otras edades. Además los menores de 16 no pueden trabajar legalmente y si bien la edad jubilatoria no es 70 perdemos muy pocos datos con esta restricvción según vimos en el TP1.

In [22]:
postulantes = postulantes[postulantes['edad'] >=16]
postulantes = postulantes[postulantes['edad'] <=70]

In [23]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693517 entries, 0 to 693703
Data columns (total 7 columns):
idpostulante       693517 non-null object
fechanacimiento    693517 non-null object
sexo               693517 non-null object
edad               693517 non-null int64
nombre             693517 non-null object
estado             693517 non-null object
nombre-estado      693517 non-null category
dtypes: category(1), int64(1), object(5)
memory usage: 37.7+ MB


In [62]:
postulantes['idpostulante'] = postulantes.drop_duplicates(subset = 'idpostulante')

In [65]:
postulantes = postulantes.dropna(subset = ['idpostulante'])

In [66]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 780019
Data columns (total 7 columns):
idpostulante       504407 non-null object
fechanacimiento    478300 non-null object
sexo               504407 non-null object
edad               504407 non-null int64
nombre             447909 non-null object
estado             447909 non-null object
nombre-estado      447909 non-null category
dtypes: category(1), int64(1), object(5)
memory usage: 27.4+ MB


In [67]:
postulantes.tail()

,idpostulante,fechanacimiento,sexo,edad,nombre,estado,nombre-estado
780015,bOjBk8b,1974-12-19,FEM,44,Terciario/Tecnico,Graduado,Terciario/Tecnico - Graduado
780016,GNZr6QK,NaN,NO_DECLARA,60,NaN,NaN,NaN
780017,bOjBkvd,1995-01-05,FEM,23,NaN,NaN,NaN
780018,5mPzjGX,1996-06-17,FEM,22,NaN,NaN,NaN
780019,jkjPrKb,NaN,NO_DECLARA,33,NaN,NaN,NaN


In [68]:
postulantes.to_csv('/home/luupesado/7506_Datos/2018/TPsDatos_1C2018/TP2/Archivos_a_usar/postulantes2')